In [ ]:
import logging
import re
import requests
import csv
from bs4 import BeautifulSoup


def get(uri):
    logging.info(f'Will query: {uri}')
    headers = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(uri, headers=headers)
    r.raise_for_status()
    r.encoding = 'UTF-8'
    return r


def get_soup(uri: str) -> BeautifulSoup:
    r = get(uri)
    return BeautifulSoup(r.text, 'html.parser')

def main():
    host = "https://www.sephora.fr"
    path = "/shop/cheveux/soin-cheveux/shampoing-c447/"
    uri = host + path
    soup = get_soup(uri)

    with open('sephora_data.csv', 'w', newline='\n',encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['ingredients', 'brand', 'name', 'price', 'quantity', 'unit'])


        expected_article_amount = soup.find('div', {'class': 'results-hits'}).text.split()[0]
        logging.info(f'Will get {expected_article_amount} of articles.')
        all_articles_uri = f'{host}{path}?srule=GoLiveSortingRule&sz={expected_article_amount}&format=page-element&start=0&on=onclickload'
        soup = get_soup(all_articles_uri)
        articles = soup.find_all('div', attrs={'class': 'product-info-wrapper'})
        logging.info(f'Got {len(articles)} articles.')
        for article in articles:
            article_product_link = article.find('a', attrs={'class': 'product-tile-link'} )['href']
            article_soup = get_soup(article_product_link)
            brand = article_soup.find('a', {'class': 'brand-link'}).text.replace('\n', '')
            name = article_soup.find('span', {'class': 'product-name product-name-bold'}).text.replace('\n', '')
            try:
                ingredients = article_soup.find('div', {'class': 'accordion-content ingredients-content'}).text.replace('\n','').split('.')[0]
                ingredients = re.sub("[\(\[].*?[\)\]]", "", ingredients)
            except Exception as e:
                logging.error("Couldn't find the ingredients")
                continue
            try:
                # unit contains the price and price the unit (go figure, I guess they are not talented enough)
                price = article_soup.find('span', {'class': 'unit'}).text.replace('\n', '').replace(' €', '')
                quantity, unit = article_soup.find('span', {'class': 'price'}).text.replace('\n', '').replace('ml', ' ml').split()
            except Exception as e:
                logging.error("Couldn't find the price or unit ")
                continue

            writer.writerow([ingredients, brand, name, price, quantity, unit])


if __name__ == '__main__':
    logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
    main()

![](sephora_data.csv)